**Illustration of text analysis for identifying potential safety incidents in patient feedback**



In [1]:
# LAUNCHING WITH VOILA LOCALLY
#1 anaconda terminal window
#2 go to the folder with the notebook to run with voila
#3 type `voila path_to_notebook optional_options --debug`
#4 close voila cmd window = CTRL+c

In [ ]:
##https://mybinder.org/v2/gh/473x/safetyIncidents_demo/master?urlpath=/voila/render/safetyIncidents_demo_v2.ipynb
#https://hub.gke2.mybinder.org/v2/gh/473x/safetyIncidents_demo/voila/render/safetyIncidents_demo_v2.ipynb
#    
# # working binder urls
#
#https://hub.gke2.mybinder.org/user/fomightez-communication_voila-93469aro/voila/render/scripts/basics.ipynb?token=yRopW3mlRO6l4yGy1IuG-Q
#
#https://hub.gke2.mybinder.org/user/dpgoldenberg-53-363fe65dd833229-eet1ar1s/voila/render/covidSimulation.ipynb?token=6kCD6eQYTGeTIWujLwjFxw   

In [2]:
# LAUNCHING VIA GITHUB & BINDER
# create environment only using pip
# pip freeze > requirements.txt
# remove any conda requirements from requirements.txt
# remove pywin32 (if present) - not needed and won't run on linux server
# push
# in my binder, add 'URL' = /voila/render/<path_to_ipynb>
# don't 'launch', but instead, copy and past the created URL

In [3]:
import ipywidgets as widgets
from ipywidgets import Textarea, Dropdown, jslink, interact, Layout
from IPython.display import  display

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.core.display import display, HTML
display(HTML("<style>.container{width:80%}</style>"))

In [4]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_lg')

In [5]:
intro = '''
Introduction: This web app illustrates the algorithm that we used to measure safety incidents in patient feedback posted online.
\nInstruction: Type text into this box, or, select an example above.
\nOutput: Below is an 'overall score' for any text in this box, and a visualization of the score for each sentence.
'''

story1 = '''
My mam was admitted to Durham hospital being sick, couldnt get her breath. A nurse knocked mam's leg getting her into bed. Mam had cellulitus, swelling to right leg, had chronic kidney disease, diabetic, copd xmas day 2011 needed beds. No ambulances.Mam sick after lunch in 24hrs, given 600mgs tramadol, nurse again knocked mams right leg same place caused tear to skin. been sick. discharged her.When mam got home, leg pouring of blood. high as a kite. should never have been discharged. Two days after, mam dizzy, had fall, fractured humerus. No pressure sores on admission, when discharged had sores left hip, right hip, heels, sacral, right elbow.Came home having to be hoisted. Tramadol 400mgs a day. Overdosed. She was sick so paramedics came took to hospital. Sepsis mam on ward six weeks. Put mam on Liverpool Care Pathway dnr (do not resusciate) without mine or mams consent.She was suffering in pain, mentally and physically. An undignified death.
'''

story2 = '''
My wife and I continue to be impressed with the entire Haematology team. The Doctor and her Clinical Asst have been nothing but supportive over the past year in stabilising my wife's condition. This team is world class and we really do appreciate we are receiving the best care, advice and support. To ease the burden on travel the Guy's team is enabling us to visit Royal Surrey haematology an equally impressive department. We are particularly impressed by the end-to-end support not just from the technical team but from the support staff who are always helpful and keep us informed with either email, phone or text messaging. People are frequently quick to be critical but VERY slow to highlight the exceptional care they receive. In writing this note we are striving to re-dress the balance in favour of the positive. Thank you so much.
'''

story3 = '''
My sister was admitted today, Sat 5th April, with severe back and chest pain. When the Triarge nurse came to assess her, she was in acute pain and couldn't breath. My sister's pain and anguish obviously irritated her and she threw her sheet down and said she would return when my sister calmed down! What arrogance! My sister was extremely distressed, had difficulty getting her breath and was very frightened. She was then left for 1 1/4 hours before she returned. I would like to say, that her arrogance could have well cost my sister her life. My sister, was later, that day diagnosed with pneumonia & and a blood clot on her lung. I am still seething with anger and intend to make an official complaint.
'''

story4 = '''
I attended A&E on three occasions & on the last of these I was moved to SAU and then onto Falcon ward. The treatment I received, both medical & personal, was excellent with nothing being too much trouble for the staff, also I couldn't fault the food with a good choice of tasty items served by helpful staff. The cleaners & the porters should also be mentioned, carrying out their work efficiently & with a smile. I have already called in & left a few small tokens of my appreciation for the staff concerned ( I hope they were enjoyed ! ). So from my viewpoint no complaints at all, great service & long may it continue !!
'''

sample_stories = {'Introduction': intro, 'Feedback 1': story1, 'Feedback 2': story2, 'Feedback 3': story3, 'Feedback 4': story4}

In [6]:
def safety_analysis(text): #, debug=False
    
    seed_terms = '''dead died disfigured dying grave incapacitated killed killing 
    maimed misdiagnosed mortally 
    murdering murderous mutilated overmedicated overmedicating perished readmitted scarred 
    succumbed traumatised'''
    seed_terms = nlp(seed_terms)
    
    doc = nlp(text)
    
    # SCORE THE WHOLE DOC
    score_label = ''
    score = round(seed_terms.similarity(doc), 3)
    score_text = [{'text': 'Overall score = '+str(score),
                      'ents': [{'start': 0, 'end': 233, 'label': score_label}],
                      'title': None}]
    score_options = {'colors': {score_label: 'rgb(250, 0, 0, '+str((score-0.4)*5)+')'}}  
    # render overall score
    displacy.render(score_text, style="ent", manual=True, options=score_options, jupyter=True)
    
    # SCORE EACH SENTENCE
    matches = []
    for sent in doc.sents:
        #print(sent)
        if sent.has_vector:
            score = round(seed_terms.similarity(sent), 2)
            match = {'start': sent.start_char,
                     'end': sent.end_char,
                     'label': str(score)}
            matches.append(match)
        else:
            pass
   
    # create display
    labels = [x['label'] for x in matches]
    # scale colors
    label_colors = [round((float(lab)-0.4)*5, 2) for lab in labels]
    # create color text
    label_colors = ['rgb(250, 0, 0, '+str(alpha)+')' for alpha in label_colors]
    # create dictionary of color dictionary
    options = {'colors': dict(zip(labels, label_colors))}
    display_text = [{"text": doc.text,
                   "ents": matches,
                   "title": None}]
    # render sentences
    displacy.render(display_text, style="ent", manual=True, options=options, jupyter=True)
    
    #if debug==True:
    #    print(f'score_options = {score_options}\n')
    #    print(f'matches = {matches}\n')
    #    print(f'labels = {labels}\n')
    #    print(f'options = {options}\n')
        

In [7]:
Dropdown.value.tag(sync=True) # necessary to sync dropdown to textarea
selection = Dropdown(value=None, 
                     description='Examples', 
                     options=sample_stories,)
text = Textarea(value=intro, 
                description='Type text', 
                layout=Layout(width="auto", height="200px"))
l = jslink((text, 'value'), (selection, 'value'))
display(selection)
interact(safety_analysis, text=text, continuous_update=False);

Dropdown(description='Examples', options={'Introduction': "\nIntroduction: This web app illustrates the algori…

interactive(children=(Textarea(value="\nIntroduction: This web app illustrates the algorithm that we used to m…